In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()
# spark.sparkContext.setLogLevel("DEBUG")

25/09/13 10:32:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
spark.stop()

In [5]:
# configs = spark.sparkContext.getConf().getAll()
# for k, v in configs:
#     print(f"{k} = {v}")

In [3]:
spark

In [4]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------+----------------+
|catalog|       namespace|
+-------+----------------+
|   demo|retail_analytics|
+-------+----------------+



In [7]:
# spark.sql("SHOW NAMESPACES IN demo").show()
# spark.sql("CREATE NAMESPACE demo.retail_analytics")
# spark.sql("USE demo.retail_analytics")

In [29]:
df = spark.read.csv("s3a://csv-input/customers.csv", header=True)

In [20]:
df.toPandas().head(5)

,customer_id,customer_address,customer_state,customer_zip_code,customer_created_date,customer_updated_date
0,00000001,604 Hampton Burg,DC,20002,2021-10-03,None
1,00000002,6588 Robert Bypass Apt. 375,RI,02869,2022-02-12,None
2,00000003,452 Gardner Gateway,FM,96944,2022-05-15,None
3,00000004,29959 Olivia Brooks,NM,88319,2022-08-01,None
4,00000005,334 Martinez Bypass,VI,00803,2023-09-07,None


In [21]:
from pyspark.sql.functions import col, coalesce, row_number
from pyspark.sql.window import Window


In [ ]:
# customer_schema = schema.get_customer_schema()

# df = extract_raw_data(sparkapp.create_spark_session(), "csv-input",  "customers.csv", schema.get_customer_schema())

window_spec = Window.partitionBy("customer_id").orderBy("updated_date")

df.withColumnsRenamed({
            "customer_address": "address",
            "customer_state": "state",
            "customer_zip_code": "zip_code_prefix",
            "customer_created_date": "created_date",
            "customer_updated_date": "updated_date",
}).withColumn(
    "row_num", row_number().over(window_spec)
).filter(col("row_num") == 1).select(
        "customer_id",
        "address",
        "state",
        "zip_code_prefix",
        "created_date",
        "updated_date"
).drop("row_num").write.mode("overwrite").partitionBy("state").saveAsTable("customers")

In [ ]:
window_spec = Window.partitionBy("customer_id").orderBy("updated_date")

df.withColumnsRenamed({
            "customer_address": "address",
            "customer_state": "state",
            "customer_zip_code": "zip_code_prefix",
            "customer_created_date": "created_date",
            "customer_updated_date": "updated_date",
}).withColumn(
    "row_num", row_number().over(window_spec)
).filter(col("row_num") == 1).select(
        "customer_id",
        "address",
        "state",
        "zip_code_prefix",
        "created_date",
        "updated_date"
).drop("row_num").write.saveAsTable("customers")


+-----------+--------------------+-----+---------------+------------+------------+
|customer_id|             address|state|zip_code_prefix|created_date|updated_date|
+-----------+--------------------+-----+---------------+------------+------------+
|   00000001|    604 Hampton Burg|   DC|          20002|  2021-10-03|        NULL|
|   00000002|6588 Robert Bypas...|   RI|          02869|  2022-02-12|        NULL|
|   00000003| 452 Gardner Gateway|   FM|          96944|  2022-05-15|        NULL|
|   00000004| 29959 Olivia Brooks|   NM|          88319|  2022-08-01|        NULL|
|   00000005| 334 Martinez Bypass|   VI|          00803|  2023-09-07|        NULL|
+-----------+--------------------+-----+---------------+------------+------------+
only showing top 5 rows



In [14]:
df.write.saveAsTable("customers")

In [12]:
df.persist()

DataFrame[customer_id: string, customer_address: string, customer_state: string, customer_zip_code: string, customer_created_date: string, customer_updated_date: string]

In [13]:
df.repartition(4)

DataFrame[customer_id: string, customer_address: string, customer_state: string, customer_zip_code: string, customer_created_date: string, customer_updated_date: string]

In [7]:
data = [("James","M"),("Michael","F"),("Jen","")]
columns = ["name","gender"]
df = spark.createDataFrame(data = data, schema = columns)
df2 = df.withColumn("gender", F.expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' ELSE 'unknown' END"))

df2.persist()

DataFrame[name: string, gender: string]

In [9]:
df2.show()

+-------+-------+
|   name| gender|
+-------+-------+
|  James|   Male|
|Michael| Female|
|    Jen|unknown|
+-------+-------+



In [10]:
df.collect()

[Row(name='James', gender='M'),
 Row(name='Michael', gender='F'),
 Row(name='Jen', gender='')]

In [ ]:
# %sql
# SELECT * FROM customers LIMIT 5

SyntaxError: invalid syntax (2995544924.py, line 2)

In [15]:
spark.sql("SELECT * FROM customers LIMIT 5").show()

+-----------+--------------------+--------------+-----------------+---------------------+---------------------+
|customer_id|    customer_address|customer_state|customer_zip_code|customer_created_date|customer_updated_date|
+-----------+--------------------+--------------+-----------------+---------------------+---------------------+
|   00000001|    604 Hampton Burg|            DC|            20002|           2021-10-03|                 NULL|
|   00000002|6588 Robert Bypas...|            RI|            02869|           2022-02-12|                 NULL|
|   00000003| 452 Gardner Gateway|            FM|            96944|           2022-05-15|                 NULL|
|   00000004| 29959 Olivia Brooks|            NM|            88319|           2022-08-01|                 NULL|
|   00000005| 334 Martinez Bypass|            VI|            00803|           2023-09-07|                 NULL|
+-----------+--------------------+--------------+-----------------+---------------------+---------------

In [16]:
spark.sql("UPDATE customers SET customer_updated_date='2025-08-11' WHERE customer_id=00000001")

DataFrame[]

In [19]:
spark.sql("SELECT * FROM customers WHERE customer_id=00000001").show()

+-----------+----------------+--------------+-----------------+---------------------+---------------------+
|customer_id|customer_address|customer_state|customer_zip_code|customer_created_date|customer_updated_date|
+-----------+----------------+--------------+-----------------+---------------------+---------------------+
|   00000001|604 Hampton Burg|            DC|            20002|           2021-10-03|           2025-08-11|
+-----------+----------------+--------------+-----------------+---------------------+---------------------+



25/09/13 10:48:52 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.7: worker lost: 172.18.0.7:33625 got disassociated
25/09/13 10:48:52 ERROR TaskSchedulerImpl: Lost executor 2 on 172.18.0.7: worker lost: 172.18.0.7:33625 got disassociated
25/09/13 10:48:52 ERROR TaskSchedulerImpl: Lost executor 1 on 172.18.0.7: worker lost: 172.18.0.7:33625 got disassociated
25/09/13 10:48:52 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_20_0 !
25/09/13 10:48:52 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_20_1 !
25/09/13 10:48:52 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_20_5 !


In [ ]:
spark.stop()

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 1871, in _run_once
    event_list = self._selector.select(timeout)
  File "